In [159]:
import json
import numpy
from collections import defaultdict
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [160]:
import os
import baseline
import pandas as pd
import numpy as np
#TEMPORARILY SUPPRESS WARNINGS
import warnings

warnings.filterwarnings("ignore")

# get the actual labels in the validation set

In [161]:
captions_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_train2017.json"
captions_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_val2017.json"
#captions_train  = json.loads(open(captions_train_path).read())

In [162]:
inst_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_train2017.json"
inst_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017.json"
#inst_train  = json.loads(open(inst_train_path).read())

In [163]:
def get_electronics_info(captions_path, inst_path):
    captions  = json.loads(open(captions_path).read())
    inst  = json.loads(open(inst_path).read())
    image_df = pd.DataFrame(inst['images'])
    annotation_df = pd.DataFrame(inst['annotations'])
    #the 'id' in image_df needs to be changed to 'image_id' in order to join with annotations_df
    renamed_image_df = image_df.copy(deep=True)
    renamed_image_df.rename(columns={'id':'image_id'}, inplace=True)
    images_and_annotations_df = annotation_df.merge(renamed_image_df,on='image_id', how='left')
    electronics_only_merged_df = images_and_annotations_df.loc[(images_and_annotations_df['category_id'] >= 72) & (images_and_annotations_df['category_id'] <= 77)]
    
    #select all images that contain electronics
    #all_images_with_electronics = list(pd.Series(electronics_only_merged_df['image_id']).unique())
    
    return(electronics_only_merged_df)

In [164]:
train_electronics = get_electronics_info(captions_train_path, inst_train_path)
val_electronics = get_electronics_info(captions_val_path, inst_val_path)

In [165]:
train_electronics.head()

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
341,9919.64620,"[308.43, 35.25, 126.4, 108.71]",72,28405,275781,0,"[[316.85, 102.67, 308.43, 63.9, 414.61, 35.25,...",http://images.cocodataset.org/train2017/000000...,2013-11-14 23:31:26,000000275781.jpg,http://farm4.staticflickr.com/3170/2601758661_...,375,3,500
342,66958.31445,"[319.77, 9.19, 319.77, 286.26]",72,28473,403107,0,"[[328.39, 10.15, 319.77, 206.42, 616.57, 295.4...",http://images.cocodataset.org/train2017/000000...,2013-11-15 15:01:19,000000403107.jpg,http://farm4.staticflickr.com/3047/3154457418_...,427,3,640
343,15215.62820,"[394.5, 42.74, 143.19, 143.19]",72,28633,203085,0,"[[414.41, 46.53, 514.93, 42.74, 513.04, 93.95,...",http://images.cocodataset.org/train2017/000000...,2013-11-15 13:00:53,000000203085.jpg,http://farm9.staticflickr.com/8065/8201625616_...,422,1,600
344,21650.08250,"[48.9, 27.33, 165.39, 149.57]",72,28660,520862,0,"[[205.66, 156.76, 179.78, 158.2, 191.28, 169.7...",http://images.cocodataset.org/train2017/000000...,2013-11-14 21:33:34,000000520862.jpg,http://farm4.staticflickr.com/3376/3631630894_...,640,2,480
345,3906.61265,"[516.13, 236.31, 63.95, 67.7]",72,28685,225913,0,"[[516.96, 242.13, 516.13, 288.64, 524.02, 291....",http://images.cocodataset.org/train2017/000000...,2013-11-14 22:16:49,000000225913.jpg,http://farm4.staticflickr.com/3152/2642920071_...,480,1,640


In [166]:
val_electronics.head()

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
48,31935.48505,"[279.11, 209.81, 206.02, 160.44]",72,29131,464476,0,"[[279.11, 370.25, 281.96, 214.56, 283.86, 209....",http://images.cocodataset.org/val2017/00000046...,2013-11-14 18:23:40,000000464476.jpg,http://farm1.staticflickr.com/24/35275654_7d93...,375,3,500
49,2293.67560,"[97.28, 202.37, 60.28, 39.91]",72,29249,68833,0,"[[100.23, 203.21, 133.68, 202.93, 155.88, 202....",http://images.cocodataset.org/val2017/00000006...,2013-11-14 16:32:09,000000068833.jpg,http://farm8.staticflickr.com/7152/6427642465_...,480,1,640
50,265862.40795,"[1.08, 1.72, 574.7, 471.39]",72,29555,433915,0,"[[388.52, 2.8, 567.17, 15.71, 574.71, 234.19, ...",http://images.cocodataset.org/val2017/00000043...,2013-11-14 19:04:41,000000433915.jpg,http://farm4.staticflickr.com/3519/3243850638_...,480,3,640
51,61254.97715,"[3.88, 0.0, 287.98, 251.13]",72,29829,149222,0,"[[4.46, 19.19, 12.67, 95.55, 16.77, 133.31, 33...",http://images.cocodataset.org/val2017/00000014...,2013-11-15 15:11:24,000000149222.jpg,http://farm1.staticflickr.com/23/33195394_22d6...,383,2,500
52,16606.20565,"[145.26, 214.29, 165.39, 107.87]",72,30246,17379,0,"[[284.76, 320.72, 267.51, 322.16, 247.37, 322....",http://images.cocodataset.org/val2017/00000001...,2013-11-15 01:16:56,000000017379.jpg,http://farm6.staticflickr.com/5123/5316824279_...,640,1,478


In [167]:
#extract electronics IDs in each image
eIDs_in_images = val_electronics.groupby('image_id')['category_id'].apply(list).to_dict()

In [168]:
#select all images that contain electronics
train_images_with_electronics = list(pd.Series(train_electronics['image_id']).unique())
val_images_with_electronics = list(pd.Series(val_electronics['image_id']).unique())

In [169]:
val_images_with_electronics[:10]

[464476, 68833, 433915, 149222, 17379, 361586, 127987, 186632, 440475, 368900]

In [170]:
len(val_images_with_electronics)

597

# get the validation images with electronics

In [171]:
val_folder_name = '/mnt/muthderd/MIDS/W210/data/val2017/'
all_val_filenames = os.listdir(val_folder_name)

In [172]:
max_file_id_length = len(all_val_filenames[0]) - 4#subtract 4 for the '.jpg' suffix

In [173]:
files_to_open = []
for image_id in val_images_with_electronics:
    file_id = str(image_id)
    zeros_to_add = max_file_id_length-len(file_id)
    filename = ('0'*zeros_to_add) + file_id + '.jpg'
    files_to_open.append(filename)

In [174]:
files_to_open[:10]

['000000464476.jpg',
 '000000068833.jpg',
 '000000433915.jpg',
 '000000149222.jpg',
 '000000017379.jpg',
 '000000361586.jpg',
 '000000127987.jpg',
 '000000186632.jpg',
 '000000440475.jpg',
 '000000368900.jpg']

# test the baseline

In [175]:
%%capture

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='yolo3_darknet53_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [176]:
pred_duration

3028.059712409973

In [177]:
all_test_annotations[:10]

,image_id,class_id,scores,1,2,3,4
0,000000464476,0.0,0.927936,70.859024,81.128845,162.252640,329.081268
1,000000464476,63.0,0.692063,376.342072,282.334595,660.263550,518.431458
2,000000464476,62.0,0.356848,376.498962,289.023132,657.539246,509.743530
3,000000464476,7.0,0.326432,194.305130,31.842560,372.927612,205.119370
4,000000464476,6.0,0.145672,194.305130,31.842560,372.927612,205.119370
5,000000464476,62.0,0.126344,206.427979,34.587311,375.598083,202.875259
6,000000464476,6.0,0.069344,27.974426,10.990707,389.384888,263.213196
7,000000464476,62.0,0.059397,374.799011,49.534012,609.736023,288.754211
8,000000464476,26.0,0.021826,72.883919,173.170853,97.293861,224.006027
9,000000464476,59.0,0.014199,376.342072,282.334595,660.263550,518.431458


In [178]:
all_test_annotations.to_csv('val_result_annotations.csv')

In [179]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [180]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [181]:
total_pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = [0, 0, 0, 0, 0, 0]
pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
label_counts = [0, 0, 0, 0, 0, 0]
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    labels = eIDs_in_images[image_id]
    for label in labels:
        total_label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        total_pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    for label in unique_labels:
        label_counts[label-72] += 1
    for pred in unique_preds:
        pred_counts[pred-72] += 1

In [182]:
total_pred_counts

[174, 150, 84, 104, 92, 110]

In [183]:
total_label_counts

[285, 224, 105, 275, 149, 249]

In [184]:
np.divide(total_pred_counts, total_label_counts)

array([0.61052632, 0.66964286, 0.8       , 0.37818182, 0.61744966,
       0.44176707])

In [185]:
pred_counts

[142, 128, 68, 77, 70, 95]

In [186]:
label_counts

[204, 176, 87, 138, 103, 205]

In [187]:
np.divide(pred_counts, label_counts)

array([0.69607843, 0.72727273, 0.7816092 , 0.55797101, 0.67961165,
       0.46341463])

In [188]:
sum(pred_counts)/sum(label_counts)

0.6352683461117196